<a href="https://colab.research.google.com/github/EyalMK/IntroductionToCloudProgramming-Tutorials/blob/main/Tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
### Imports
import ipywidgets as widgets
import os
from IPython.display import display, clear_output
from google.colab import drive

In [18]:
### Utilities Class
class Utilities:
  def __init__(self):
    return

  ### Function to read a given file into an array line by line
  def import_file_line_by_line(self, file_descriptor):
      return [l for l in (line.strip() for line in file_descriptor) if l]

  ### Function to parse student info
  def parse_student_info(self, student_info):
      parts = student_info.split(", ", 4)

      # Ensure parts has the correct length
      if len(parts) < 5:
          parts.append("")

      name, email, courses, url, favorite_program = parts
      courses = courses.strip("()")
      return name, email, courses, url, favorite_program

  ### Function to parse student email
  def parse_student_email(self, student_info):
      _, email, _, _, _ = self.parse_student_info(student_info)
      return email

  ### Remove a line from file
  def remove_line_from_file(self, file_name, line_to_remove):
      # Read all lines from the file
      with open(file_name, 'r') as file:
          lines = file.readlines()

      # Remove the line that matches the given content
      lines = [line for line in lines if line.strip() != line_to_remove.strip()]

      # Write the remaining lines back to the file
      with open(file_name, 'w') as file:
          file.writelines(lines)

In [19]:
### App Class
class Tutorial1:
  def __init__(self, file_name):
    self.file_name = file_name
    self.name_label = widgets.Label(value="Name: ")
    self.email_label = widgets.Label(value="Email: ")
    self.courses_label = widgets.Label(value="Courses: ")
    self.url_label = widgets.Label(value="URL: ")
    self.program_label = widgets.Label(value="Favorite Program: ")
    self.output = widgets.Output()
    self.utils = Utilities()

    self.mount_drive()

    with self.output:
      with open(file_name, 'r') as file:
        lines = self.utils.import_file_line_by_line(file)
        self.dropdown_list = widgets.Dropdown(
            options = lines,
            description = "Students:",
            disabled = False,
          )
        self.input_field = widgets.Text(
            value="",
            description='Favorite Program:',
            disabled=False
          )
        self.button = widgets.Button(
            description='Submit',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Submit changes',
            icon='check' # (FontAwesome names without the `fa-` prefix)
          )

  ### Mount point to Google Drive and access MyDrive
  def mount_drive(self):
    drive.mount("/content/drive", force_remount=True)
    os.chdir("/content/drive/MyDrive")

  ### Dropdown list on selection event
  def list_selection(self, change):
    with self.output:
      clear_output()
      selected_student = change['new']
      selected_email = self.utils.parse_student_email(selected_student)
      matching_option = next((opt for opt in self.dropdown_list.options if selected_email in opt), None)
      with open(self.file_name, 'r') as file:
          lines = self.utils.import_file_line_by_line(file)
          if matching_option:
            name, email, courses, url, favorite_program = self.utils.parse_student_info(selected_student)
            self.name_label.value = f"Name: {name}"
            self.email_label.value = f"Email: {email}"
            self.courses_label.value = f"Courses: {courses}"
            self.url_label.value = f"URL: {url}"
            self.program_label.value = f"Favorite Program: {favorite_program}"
            self.input_field.value = favorite_program
          else:
            print("Something went wrong... Student not found.")

  ### Submit button on click event
  def handle_submit(self, sender):
    with self.output:
      with open(self.file_name, 'a') as file:
        selected_value = self.dropdown_list.value
        name, email, courses, url, _ = self.utils.parse_student_info(selected_value)
        new_value = f"{name}, {email}, {courses}, {url}, {self.input_field.value}"

        self.utils.remove_line_from_file(self.file_name, self.dropdown_list.value)
        file.write(f"\n{name}, {email}, {courses}, {url}, {self.input_field.value}")
        self.program_label.value = f"Favorite Program: {self.input_field.value}"
      with open(self.file_name, 'r') as readfile:
        self.dropdown_list.options = self.utils.import_file_line_by_line(readfile)
        self.dropdown_list.value = new_value

  def display(self):
    ### Display widgets and setup event listener
    self.dropdown_list.observe(self.list_selection, names='value')
    self.button.on_click(self.handle_submit)
    display(self.dropdown_list, self.input_field, self.button, self.name_label, self.email_label, self.courses_label, self.url_label, self.program_label, self.output)
    self.list_selection({'new': self.dropdown_list.value})

In [20]:
### Main Instance
if __name__ == '__main__':
  app = Tutorial1("students.txt")
  app.display()

Mounted at /content/drive


Dropdown(description='Students:', options=('Adir David, Adir.David@e.braude.ac.il, Introduction to Cloud Compu…

Text(value='', description='Favorite Program:')

Button(button_style='success', description='Submit', icon='check', style=ButtonStyle(), tooltip='Submit change…

Label(value='Name: ')

Label(value='Email: ')

Label(value='Courses: ')

Label(value='URL: ')

Label(value='Favorite Program: ')

Output()